# 1. 필요한 패키지 불러오기

In [ ]:
!pip install konlpy

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Komoran
from time import time
import json

# 2. get_prediction 함수

1. 이 함수는 테스트에 해당하는 csv 파일 경로를 매개변수로 가집니다.
2. 이 함수는 미리 만들어진 모델을 불러와 예측을 수행합니다. 
3. 이 함수는 예측치를 불러온 csv 파일에 넣습니다. 이 때 예측치에 해당하는 column name은 smishing 입니다.
4. 이 함수는 데이터프레임을 리턴합니다.

In [3]:
def get_prediction(test_file_path):
    '''
    Arg.: String
    Return: Pandas DataFrame
    '''
    
    '''1. 테스트 파일 불러오기'''
    public_test = pd.read_csv('../0_Data/public_test.csv')
    
    '''2. 모델 불러오기'''
    new_model = load_model('../1_Model/np7_random0_tf8_val0.05.h5')
    
    '''3. 예측 전 필요한 사항 진행하기'''
    test = public_test['text']
    
    new_test = []
    kom = Komoran(userdic='../user_pos_dic.txt') #사용자 사전 사용시 괄호 안에 userdic='경로/user_pos_dic.txt'
    for i, document in enumerate(test):
        pos = kom.pos(document)
        clean_words = []
        for p in pos:
            if p[1] == 'NNP' or p[1] == 'NNG' or p[1] == 'SL':
                clean_words.append(p[0])
        document = ' '.join(clean_words)
        new_test.append(document)
    
    #Train 데이터에서 fit 시킨 tokenizer word index를 불러오기
    tokenizer = Tokenizer()
    
    with open('../wordIndex.json') as json_file:
        word_index = json.load(json_file)
        tokenizer.word_index = word_index
    
    X_test = new_test
    X_test_1 = tokenizer.texts_to_sequences(X_test)
    X_test_2 = pad_sequences(X_test_1, maxlen=400)
    
    '''4. 예측 진행하기'''
    y_pred = new_model.predict_proba(X_test_2)
    
    '''5. 예측치 데이터프레임에 합치기'''
    public_test['smishing']=y_pred
    return public_test

### a) 인풋데이터 예시

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>id</th>
      <th>year_month</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>340000</td>
      <td>2019-01</td>
      <td>XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>340001</td>
      <td>2019-01</td>
      <td>긴급 안내  XXX은행 가락동 지점  - 헬리오XXX 기본XXX    대출이자를 ...</td>
    </tr>
    <tr>
      <th>2</th>
      <td>340002</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>340003</td>
      <td>2019-01</td>
      <td>XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>340004</td>
      <td>2019-01</td>
      <td>XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다.  고...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>340005</td>
      <td>2019-01</td>
      <td>기기분좋게해해피하게년년중내내1월의바램과 소원 쭈욱겨울의 꽃 1월행복으로 마감되는눈을...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>340006</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요.XXX은행 XXX지점 VIP팀장 XXX입니다.제가 이번에 ...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>340007</td>
      <td>2019-01</td>
      <td>(광고)XXX 고객님XXX외화 환전 서비스 안내XXX은행 고현종합금융센터입니다.외화...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>340008</td>
      <td>2019-01</td>
      <td>XXX 고객님우리가 쉽게 지나치는세잎클로버의 꽃 말이행복이라는 사실 알고계셨나요.늘...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>340009</td>
      <td>2019-01</td>
      <td>- 법인계좌 압류등록 안내    2018.11.23  3115790원 한울상사   ...</td>
    </tr>
  </tbody>
</table>

### b) 아웃풋데이터 예시

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>id</th>
      <th>year_month</th>
      <th>text</th>
      <th>smishing</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>340000</td>
      <td>2019-01</td>
      <td>XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...</td>
      <td>0.200</td>
    </tr>
    <tr>
      <th>1</th>
      <td>340001</td>
      <td>2019-01</td>
      <td>긴급 안내  XXX은행 가락동 지점  - 헬리오XXX 기본XXX    대출이자를 ...</td>
      <td>0.413</td>
    </tr>
    <tr>
      <th>2</th>
      <td>340002</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...</td>
      <td>0.560</td>
    </tr>
    <tr>
      <th>3</th>
      <td>340003</td>
      <td>2019-01</td>
      <td>XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...</td>
      <td>0.990</td>
    </tr>
    <tr>
      <th>4</th>
      <td>340004</td>
      <td>2019-01</td>
      <td>XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다.  고...</td>
      <td>0.010</td>
    </tr>
    <tr>
      <th>5</th>
      <td>340005</td>
      <td>2019-01</td>
      <td>기기분좋게해해피하게년년중내내1월의바램과 소원 쭈욱겨울의 꽃 1월행복으로 마감되는눈을...</td>
      <td>0.430</td>
    </tr>
    <tr>
      <th>6</th>
      <td>340006</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요.XXX은행 XXX지점 VIP팀장 XXX입니다.제가 이번에 ...</td>
      <td>0.120</td>
    </tr>
    <tr>
      <th>7</th>
      <td>340007</td>
      <td>2019-01</td>
      <td>(광고)XXX 고객님XXX외화 환전 서비스 안내XXX은행 고현종합금융센터입니다.외화...</td>
      <td>0.320</td>
    </tr>
    <tr>
      <th>8</th>
      <td>340008</td>
      <td>2019-01</td>
      <td>XXX 고객님우리가 쉽게 지나치는세잎클로버의 꽃 말이행복이라는 사실 알고계셨나요.늘...</td>
      <td>0.110</td>
    </tr>
    <tr>
      <th>9</th>
      <td>340009</td>
      <td>2019-01</td>
      <td>- 법인계좌 압류등록 안내    2018.11.23  3115790원 한울상사   ...</td>
      <td>0.200</td>
    </tr>
  </tbody>
</table>

In [ ]:
%%time
get_prediction('../0_Data/public_test.csv').head(20)